In [ ]:
 Cross-Sectional Measures

In [15]:
import numpy as np
class MarginalExpectedShortfall:
    def __init__(self, firm_returns: np.ndarray, market_returns: np.ndarray) -> None:
        assert firm_returns.shape == market_returns.shape
        self.firm_returns = firm_returns
        self.market_returns = market_returns

    def estimate(self, q: float = 0.05) -> float:
        low_threshold = np.percentile(self.market_returns, q * 100)
        worst_days = np.argwhere(self.market_returns < low_threshold)
        return np.mean(self.firm_returns[worst_days])

In [19]:
import numpy as np
class SystemicExpectedShortfall:
    def __init__(
        self,
        mes_training_sample: np.ndarray,
        lvg_training_sample: np.ndarray,
        ses_training_sample: np.ndarray,
        mes_firm: float,
        lvg_firm: float,
    ) -> None:
        self.mes = mes_training_sample
        self.lvg = lvg_training_sample
        self.ses = ses_training_sample
        self.mes_firm = mes_firm
        self.lvg_firm = lvg_firm

    def estimate(self) -> float:
        n_firms = self.mes.shape

        data = np.vstack([np.ones(n_firms), self.mes, self.lvg]).T
        betas = np.linalg.lstsq(data, self.ses, rcond=None)[0]
        _, b, c = betas
        ses = (b * self.mes_firm + c * self.lvg_firm) / (b + c)
        return ses

In [20]:
import numpy as np
mes_training_sample = np.array([-0.023, -0.07, 0.01])
lvg_training_sample = np.array([1.8, 1.5, 2.2])
ses_training_sample = np.array([0.3, 0.4, -0.2])
mes_firm = 0.04
lvg_firm = 1.7
ses = SystemicExpectedShortfall(mes_training_sample, lvg_training_sample, ses_training_sample, mes_firm, lvg_firm)
ses.estimate()


-0.3334075723830676

In [21]:
from numpy.random import RandomState

rng = RandomState(0)
firm_returns = rng.normal(0, 1, 100)
mkt_returns = rng.normal(0, 1, 100)
mes = MarginalExpectedShortfall(firm_returns, mkt_returns)
res = mes.estimate()
res

0.13494025343324562

In [ ]:
function mes = marginal_expected_shortfall(firm_returns, market_returns)
%
% Calculates the marginal expected shortfall of a firm.
%
% PARAMETERS:
% firm_returns - The time series of returns for the firm.
% market_returns - The time series of returns for the market.

In [23]:
import numpy as np
from typing import Union
class SRISK:
    def __init__(
        self,
        firm_returns: np.ndarray,
        market_returns: np.ndarray,
        W: Union[float, np.ndarray],
        D: Union[float, np.ndarray],
    ) -> None:
        if len(firm_returns.shape) == 1:
            # Single firm
            n_firms = 1
            n_days = len(firm_returns)
            
        else:
            # Multple firms
            n_days, n_firms = firm_returns.shape

        self.firm_returns = firm_returns
        self.market_returns = market_returns
        self.W = W
        self.D = D
        self.n_firms = n_firms
        self.n_days = n_days

    def estimate(
        self,
        k=0.08,
        lrmes_h=22,
        lrmes_S=10000,
        lrmes_C=-0.1,
        lrmes_random_seed=42,
        aggregate_srisk=False,
    ) -> Union[np.ndarray, float]:
        market_returns = self.market_returns
        if self.n_firms == 1:
            lrmes = LongRunMarginalExpectedShortfall(
                self.firm_returns, market_returns
            ).estimate(lrmes_h, lrmes_S, lrmes_C, lrmes_random_seed)
        else:
            lrmes = np.empty(self.n_firms)
            for i in range(self.n_firms):
                firm_returns = self.firm_returns[:, i]
                lrmes[i] = LongRunMarginalExpectedShortfall(
                    firm_returns, market_returns
                ).estimate(lrmes_h, lrmes_S, lrmes_C, lrmes_random_seed)

        lvg = (self.D + self.W) / self.W
        srisk = self.W * (k * lvg + (1 - k) * lrmes - 1)
        if not aggregate_srisk:
            return srisk
        else:
            return np.sum(srisk.clip(min=0.0))